<a href="https://colab.research.google.com/github/auqa5325/AUQA-colab/blob/main/Copy_of_AUQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install boto3 -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 6.0 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

AWS_ACCESS_KEY_ID     = userdata.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = userdata.get("AWS_SECRET_ACCESS_KEY")
#AWS_SESSION_TOKEN     = userdata.get("AWS_SESSION_TOKEN")  # may be None
AWS_REGION = userdata.get("AWS_REGION")

os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
os.environ["AWS_REGION"] = AWS_REGION

print("✅ Credentials set. Region:", AWS_REGION)


✅ Credentials set. Region: ap-south-1


In [3]:
import boto3, os

REGION = os.environ["AWS_REGION"]
session = boto3.Session(region_name=REGION)
bedrock         = session.client("bedrock")
bedrock_runtime = session.client("bedrock-runtime")
print("✅ boto3 session initialized in:", session.region_name)


✅ boto3 session initialized in: ap-south-1


In [4]:
import boto3
s = boto3.Session(region_name="ap-south-1")
print(s.client("sts").get_caller_identity())   # must print Account, Arn
creds = s.get_credentials().get_frozen_credentials()
print("AccessKey:", creds.access_key[:4], "HasToken:", bool(creds.token))


{'UserId': 'AIDA4L4FNUDYI73ZUG6BD', 'Account': '850146468080', 'Arn': 'arn:aws:iam::850146468080:user/Anna_university_L1', 'ResponseMetadata': {'RequestId': '47d69c2b-c3ec-4e7f-836f-2c1f10053209', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '47d69c2b-c3ec-4e7f-836f-2c1f10053209', 'x-amz-sts-extended-request-id': 'MTphcC1zb3V0aC0xOjE3NTY1NDg4MzQxNDA6Ujp3Y0FQd1kzUA==', 'content-type': 'text/xml', 'content-length': '415', 'date': 'Sat, 30 Aug 2025 10:13:54 GMT'}, 'RetryAttempts': 0}}
AccessKey: AKIA HasToken: False


# output dict structure :
* id → unique identifier for this response (good for
logging/debugging).

* type → "message" → tells you this is a message object.

* role → "assistant" → the speaker role (assistant vs. user).

* model → which model gave this reply (claude-3-sonnet-20240229).

* content → a list of parts that make up the response.

* stop_reason → why the model stopped (e.g., end_turn, max_tokens).

* stop_sequence → custom sequence that stopped generation (here it’s None).

* usage → token usage info (handy for cost + rate limits).

In [ ]:
import json
model_id = "mistral.mixtral-8x7b-instruct-v0:1"
prompt = "explain tcp ip?"

body = {
    "prompt": prompt,
    "max_tokens": 100,
}

resp = bedrock_runtime.invoke_model(modelId=model_id, body=json.dumps(body))
output = json.loads(resp["body"].read())
print(output["outputs"][0]["text"])


In [4]:
import boto3
session = boto3.Session(region_name="ap-south-1")
sts = session.client("sts")
print(sts.get_caller_identity())  # should return Account, Arn, UserId


{'UserId': 'AIDA4L4FNUDYI73ZUG6BD', 'Account': '850146468080', 'Arn': 'arn:aws:iam::850146468080:user/Anna_university_L1', 'ResponseMetadata': {'RequestId': '0318db39-9d37-4c6e-bb25-6767026e89f2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0318db39-9d37-4c6e-bb25-6767026e89f2', 'x-amz-sts-extended-request-id': 'MTphcC1zb3V0aC0xOjE3NTY0NTkzNTE3NDk6UjpJd1BrYlNOVg==', 'content-type': 'text/xml', 'content-length': '415', 'date': 'Fri, 29 Aug 2025 09:22:31 GMT'}, 'RetryAttempts': 0}}


{'outputs': [{'text': '\n\nTCP/IP is a suite of protocols that defines the Internet. Originally designed for the UNIX operating system. The TCP/IP protocol suite is a four-layer model that provides end-to-end connectivity specifying how data should be packetized, addressed, transmitted, routed, and received.\n\nThe four layers of the TCP/IP model are:\n\n1. The Application Layer: This is the topmost layer of the TCP/IP', 'stop_reason': 'length'}]}

In [5]:
!pip -q install opensearch-py requests-aws4auth

from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3

# --- CONFIG ---
host   = "https://search-test1-annauniv-pcx3f52wxykhpd4md6v4bjeqdy.ap-south-1.es.amazonaws.com"  # OpenSearch domain endpoint
region = "ap-south-1"
service = "es"

# --- AWS SigV4 Auth ---
session = boto3.Session()
credentials = session.get_credentials()
awsauth = AWS4Auth(
    credentials.access_key,
    credentials.secret_key,
    region,
    service,
    session_token=credentials.token
)

# --- OpenSearch Client ---
client = OpenSearch(
    hosts=[host],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)

# --- TEST: Get cluster info ---
print(client.info())


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.5/371.5 kB 7.4 MB/s eta 0:00:00
{'name': 'd69eb8cacc6e2cf42750a5f4788db327', 'cluster_name': '850146468080:test1-annauniv', 'cluster_uuid': 'qhkHo9X9Suuw8REB9uInGQ', 'version': {'distribution': 'opensearch', 'number': '2.19.0', 'build_type': 'tar', 'build_hash': 'unknown', 'build_date': '2025-07-24T06:15:41.026838036Z', 'build_snapshot': False, 'lucene_version': '9.12.1', 'minimum_wire_compatibility_version': '7.10.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'The OpenSearch Project: https://opensearch.org/'}


In [83]:
"""
index_name = "test-auqa"

index_body = {
    "settings": {
        "index": {
            "knn": True,
            "knn.algo_param.ef_search": 512,
            "number_of_shards": 5,
            "number_of_replicas": 1
        }
    },
    "mappings": {
        "properties": {
            # main chunk text
            "chunk_text": {
                "type": "text",
                "fields": {
                    "keyword": {"type": "keyword", "ignore_above": 256}
                }
            },

            # ids & names
            "course_id": {
                "type": "text",
                "fields": {
                    "keyword": {"type": "keyword", "ignore_above": 256}
                }
            },

            "filename": {
                "type": "text",
                "fields": {
                    "keyword": {"type": "keyword", "ignore_above": 256}
                }
            },

            # page number
            "page_no": { "type": "long" },



            # vector index used for ANN similarity search
            "vector_field": {
                "type": "knn_vector",
                "dimension": 1024,
                "method": {
                    "name": "hnsw",
                    "space_type": "cosinesimil",
                    "engine": "nmslib",
                    "parameters": { "ef_construction": 512, "m": 16 }
                }
            }
        }
    }
}

# recreate index
if client.indices.exists(index=index_name):
    print(f"Index '{index_name}' already exists. Deleting and recreating...")
    client.indices.delete(index=index_name)

client.indices.create(index=index_name, body=index_body)
print(f"✅ Index '{index_name}' created successfully!")
"""

SyntaxError: incomplete input (ipython-input-2645953989.py, line 64)

{'test-annauniv': {'aliases': {}, 'mappings': {'properties': {'course_id': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'embedding': {'type': 'float'}, 'metadata': {'properties': {'page': {'type': 'long'}, 'source': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}}, 'page_no': {'type': 'long'}, 'text': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'vector_field': {'type': 'knn_vector', 'dimension': 1024, 'method': {'engine': 'nmslib', 'space_type': 'cosinesimil', 'name': 'hnsw', 'parameters': {'ef_construction': 512, 'm': 16}}}}}, 'settings': {'index': {'replication': {'type': 'DOCUMENT'}, 'refresh_interval': '1s', 'number_of_shards': '5', 'knn.algo_param': {'ef_search': '512'}, 'provided_name': 'test-annauniv', 'knn': 'true', 'creation_date': '1756144596121', 'number_of_replicas': '1', 'uuid': 'rumasDZaQC6xBy4G48v1KQ', 'version': {'created': '136407827'}}}}}



In [10]:
import boto3, time, re

BUCKET = "anna-univ-qna"
DOCUMENT = "Textbooks/SPM.pdf"
REGION = "ap-south-1"
COURSE_ID = "CS6022"   # <-- set this per textbook
FILENAME = "SPM.pdf"

textract = boto3.client("textract", region_name=REGION)

# --- Start async Textract job ---
start = time.time()
response = textract.start_document_text_detection(
    DocumentLocation={"S3Object": {"Bucket": BUCKET, "Name": DOCUMENT}}
)
print("FILENAME =",FILENAME)
job_id = response["JobId"]
print(f"✅ Started Textract JobId: {job_id}")

# --- Wait for completion ---
while True:
    result = textract.get_document_text_detection(JobId=job_id)
    status = result["JobStatus"]
    if status in ["SUCCEEDED", "FAILED"]:
        break
    time.sleep(5)

elapsed = time.time() - start
if status == "FAILED":
    raise Exception("❌ Textract job failed!")

pages = result["DocumentMetadata"]["Pages"]
print(f"📄 Pages: {pages}")
print(f"⏱️ Time taken: {elapsed:.2f} sec")
print(f"💰 Estimated cost: ${(pages/1000)*1.5:.4f}")

# --- Collect page-wise text into docs with metadata ---
page_texts = {}
next_token = None

while True:
    if next_token:
        result = textract.get_document_text_detection(JobId=job_id, NextToken=next_token)
    else:
        result = textract.get_document_text_detection(JobId=job_id)

    for block in result["Blocks"]:
        if block["BlockType"] == "LINE":
            page_no = block["Page"]
            page_texts.setdefault(page_no, []).append(block["Text"])

    next_token = result.get("NextToken")
    if not next_token:
        break

# Build document objects (ready to send to Titan for embeddings)
docs = []
for page_no, lines in page_texts.items():
    chunk_text = "\n".join(lines)
    doc = {
        "chunk_text": chunk_text,
        "course_id": COURSE_ID,
        "filename": FILENAME,
        "page_no": page_no
    }
    docs.append(doc)

# --- Preview first 5 pages ---
print("\n📑 First 5 pages extracted:")
for d in docs[:5]:
    print(f"\nPage {d['page_no']} | Course: {d['course_id']} | File: {d['filename']}")
    print(d["chunk_text"][:400], "...")


FILENAME = SPM.pdf
✅ Started Textract JobId: ce41d655412d458abda1771970d243aee8b1fcb47f07c25ecdfed15620863448
📄 Pages: 396
⏱️ Time taken: 231.58 sec
💰 Estimated cost: $0.5940

📑 First 5 pages extracted:

Page 1 | Course: CS6022 | File: SPM.pdf
BoB HUGHES AND MIKE COTTERELL
Software
Project
Management Second Edition
Tistimating
www.mcgraw-hill.co.uk/hughes ...

Page 2 | Course: CS6022 | File: SPM.pdf
Software Project Management
(Second Edition) ...

Page 3 | Course: CS6022 | File: SPM.pdf
Software Project Management
(Second Edition)
Bob Hughes and Mike Cotterell,
School of Information Management, University of Brighton
The McGraw-Hill Companies
London
Burr Ridge, IL
New York
St Louis
San Francisco
Auckland
Bogotá Caracas
Lisbon
Madrid
Mexico
Milan
Montreal
New Delhi
Panama
Paris
San Juan
São Paulo
Singapore
Tokyo
Toronto ...

Page 4 | Course: CS6022 | File: SPM.pdf
Published by
McGraw-Hill Publishing Company
SHOPPENHANGERS ROAD, MAIDENHEAD, BERKSHIRE, SL6 2QL, ENGLAND
Telephone: +44(o) 

In [11]:
import boto3, json, time

# --- Config ---
REGION = "ap-south-1"
MODEL_ID = "amazon.titan-embed-text-v2:0"
PRICE_PER_1K = 0.000024  # USD per 1k tokens

# Assume docs is already built from Textract step
# docs = [
#   {"chunk_text": "...", "course_id":"CS101", "filename":"SPM.pdf", "page_no":1},
#   ...
# ]

bedrock_rt = boto3.client("bedrock-runtime", region_name=REGION)

all_embeddings = []
total_tokens = 0

start = time.time()

for d in docs:
    body = json.dumps({
        "inputText": d["chunk_text"]
    })

    resp = bedrock_rt.invoke_model(modelId=MODEL_ID, body=body)
    result = json.loads(resp["body"].read())

    # Titan Embedding returns: { "embedding": [...], "inputTextTokenCount": N }
    vector = result["embedding"]
    tokens = result.get("inputTextTokenCount", len(d["chunk_text"].split()))  # fallback

    d["vector_field"] = vector
    d["tokens"] = tokens

    all_embeddings.append(d)

    total_tokens += tokens
    print(f"📄 Page {d['page_no']}: {tokens} tokens")

elapsed = time.time() - start
cost = (total_tokens / 1000) * PRICE_PER_1K

print("\n⏱️ Total time:", round(elapsed, 2), "seconds")
print("📊 Total tokens:", total_tokens)
print(f"💰 Estimated embedding cost: ${cost:.6f}")


📄 Page 1: 37 tokens
📄 Page 2: 9 tokens
📄 Page 3: 96 tokens
📄 Page 4: 345 tokens
📄 Page 5: 14 tokens
📄 Page 6: 393 tokens
📄 Page 7: 424 tokens
📄 Page 8: 393 tokens
📄 Page 9: 393 tokens
📄 Page 10: 253 tokens
📄 Page 11: 517 tokens
📄 Page 12: 482 tokens
📄 Page 13: 235 tokens
📄 Page 14: 460 tokens
📄 Page 15: 551 tokens
📄 Page 16: 373 tokens
📄 Page 17: 575 tokens
📄 Page 18: 463 tokens
📄 Page 19: 400 tokens
📄 Page 20: 529 tokens
📄 Page 21: 457 tokens
📄 Page 22: 410 tokens
📄 Page 23: 645 tokens
📄 Page 24: 322 tokens
📄 Page 25: 558 tokens
📄 Page 26: 550 tokens
📄 Page 27: 411 tokens
📄 Page 28: 588 tokens
📄 Page 29: 457 tokens
📄 Page 30: 558 tokens
📄 Page 31: 365 tokens
📄 Page 32: 603 tokens
📄 Page 33: 403 tokens
📄 Page 34: 273 tokens
📄 Page 35: 515 tokens
📄 Page 36: 542 tokens
📄 Page 37: 588 tokens
📄 Page 38: 553 tokens
📄 Page 39: 520 tokens
📄 Page 40: 540 tokens
📄 Page 41: 276 tokens
📄 Page 42: 443 tokens
📄 Page 43: 369 tokens
📄 Page 44: 600 tokens
📄 Page 45: 532 tokens
📄 Page 46: 529 tokens
📄 

In [12]:
import hashlib, re

def normalize_text(s: str) -> str:
    """Normalize text to avoid minor whitespace/case changes causing new IDs."""
    s = s.strip()
    s = re.sub(r"\s+", " ", s)
    return s.lower()

def make_chunk_id(course_id: str, chunk_text: str) -> str:
    """Deterministic ID based on course_id + chunk_text."""
    norm = normalize_text(chunk_text)
    raw = f"{course_id}|{norm}"
    return hashlib.blake2b(raw.encode("utf-8"), digest_size=16).hexdigest()


In [19]:
from opensearchpy.helpers import parallel_bulk
index_name = "test-auqa"
def doc_to_action(doc):
    doc_id = make_chunk_id(doc["course_id"], doc["chunk_text"])
    return {
        "_op_type": "index",          # overwrite if exists
        "_index": index_name,
        "_id": doc_id,
        "_source": {
            "chunk_text": doc["chunk_text"],
            "course_id": doc["course_id"],
            "filename": doc["filename"],
            "page_no": doc["page_no"],
            "vector_field": doc["vector_field"]  # Titan embedding (1024 floats)
        }
    }

actions = (doc_to_action(d) for d in all_embeddings)

for ok, result in parallel_bulk(client, actions, thread_count=1, chunk_size=100):
    if not ok:
        print("❌ Failed:", result)
print("✅ Bulk upsert finished.")


✅ Bulk upsert finished.


In [52]:
MODEL_ID = "amazon.titan-embed-text-v2:0"

bedrock_rt = boto3.client("bedrock-runtime", region_name=REGION)
def get_titan_embedding(text: str):
    body = json.dumps({"inputText": text})
    resp = bedrock_rt.invoke_model(modelId=MODEL_ID, body=body)
    result = json.loads(resp["body"].read())
    return result["embedding"], result.get("inputTextTokenCount", None)

query_text = "Cost-benefit evaluation techniques"
query_vector, token_count = get_titan_embedding(query_text)

print(f"✅ Query embedded: {len(query_vector)}-dim vector | Tokens: {token_count}")


✅ Query embedded: 1024-dim vector | Tokens: 6


In [66]:
TOP_N = 20
BM25_WEIGHT = 0.3
VECTOR_WEIGHT = 0.7

# --- Run BM25 ---
bm25_resp = client.search(
    index=index_name,
    body={"size": TOP_N, "query": {"match": {"chunk_text": query_text}}}
)
bm25_hits = bm25_resp["hits"]["hits"]

# --- Run Vector ---
vector_resp = client.search(
    index=index_name,
    body={
        "size": TOP_N,
        "query": {
            "knn": {"vector_field": {"vector": query_vector, "k": TOP_N}}
        }
    }
)
vector_hits = vector_resp["hits"]["hits"]

# --- Normalize scores ---
def normalize_scores(hits):
    scores = [h["_score"] for h in hits]
    if not scores:
        return {}
    min_s, max_s = min(scores), max(scores)
    if min_s == max_s:
        return {h["_id"]: 1.0 for h in hits}
    return {h["_id"]: (h["_score"] - min_s) / (max_s - min_s) for h in hits}

bm25_norm = normalize_scores(bm25_hits)
vector_norm = normalize_scores(vector_hits)

# --- Combine ---
combined = {}
for h in bm25_hits + vector_hits:
    _id = h["_id"]
    src = h["_source"]
    bm25_s = bm25_norm.get(_id, 0.0)
    vec_s = vector_norm.get(_id, 0.0)
    hybrid_score = BM25_WEIGHT * bm25_s + VECTOR_WEIGHT * vec_s
    combined[_id] = {
        "hybrid_score": hybrid_score,
        "bm25_score": bm25_s,
        "vector_score": vec_s,
        "source": src
    }

results = sorted(combined.values(), key=lambda x: x["hybrid_score"], reverse=True)

# --- Print top 5 results ---
print("\n🔎 Manual Hybrid Results:")
for r in results[:]:
    src = r["source"]
    print(f"Hybrid={r['hybrid_score']:.3f} | BM25={r['bm25_score']:.3f} | Vec={r['vector_score']:.3f}")
    print(f"Page={src['page_no']} | Course={src['course_id']} | File={src['filename']}")
    print(src["chunk_text"][:10], "...\n")



🔎 Manual Hybrid Results:
Hybrid=0.919 | BM25=0.729 | Vec=1.000
Page=55 | Course=CS6022 | File=SPM.pdf
3.6 COST-B ...

Hybrid=0.734 | BM25=0.580 | Vec=0.801
Page=53 | Course=CS6022 | File=SPM.pdf
3.4 COST-B ...

Hybrid=0.557 | BM25=0.312 | Vec=0.663
Page=57 | Course=CS6022 | File=SPM.pdf
3.6 COST-B ...

Hybrid=0.522 | BM25=0.681 | Vec=0.454
Page=61 | Course=CS6022 | File=SPM.pdf
3.6 COST-B ...

Hybrid=0.467 | BM25=0.267 | Vec=0.552
Page=59 | Course=CS6022 | File=SPM.pdf
3.6 COST-B ...

Hybrid=0.466 | BM25=1.000 | Vec=0.237
Page=7 | Course=CS6022 | File=SPM.pdf
viii
CONTE ...

Hybrid=0.442 | BM25=0.529 | Vec=0.405
Page=52 | Course=CS6022 | File=SPM.pdf
40
CHAPTER ...

Hybrid=0.410 | BM25=0.521 | Vec=0.363
Page=63 | Course=CS6022 | File=SPM.pdf
3.7 RISK E ...

Hybrid=0.347 | BM25=0.803 | Vec=0.152
Page=49 | Course=CS6022 | File=SPM.pdf
Chapter 3
 ...

Hybrid=0.277 | BM25=0.923 | Vec=0.000
Page=155 | Course=CS6022 | File=SPM.pdf
7.7 EVALUA ...

Hybrid=0.274 | BM25=0.505 | Vec=0.175
Page=6

In [71]:
# Collect top 5 page chunks into one string
context_text = "\n\n".join([r["source"]["chunk_text"] for r in results[:]])
ques_query = query_text   # the query you used
no = 20


In [74]:
prompt = f"""
You are an AI assistant specialized in **question generation and analysis**.

Your goals:
1. Generate insightful and generic questions based on the given context and user query.
   - Do not copy sentences from the context.
   - Make questions broad and meaningful, not text-specific.
   - Generate at least **{no} questions**.
2. For each generated question, classify it by:
   - **Difficulty level**:
       * Easy → simple recall/basic understanding (1-sentence answers).
       * Medium → requires moderate understanding, application, or 2–3 steps of reasoning.
       * Hard → requires deep understanding, critical thinking, multi-step reasoning.
   - **Bloom's taxonomy level**:
       * Choose one of: Remember, Understand, Apply, Analyze, Evaluate, Create.

### Input
Context:
{context_text}

User Query (Focus Topic):
{ques_query}

### Output
Return the results in **valid JSON** format, as a list of objects.
Each object must look like this:
{{
  "question": "...",
  "difficulty": "...",
  "blooms_level": "..."
}}

### Questions:
"""


In [75]:
import boto3, json, time

REGION = "ap-south-1"
PROFILE_ARN = "arn:aws:bedrock:ap-south-1:850146468080:inference-profile/apac.anthropic.claude-3-5-sonnet-20241022-v2:0"

bedrock_rt = boto3.client("bedrock-runtime", region_name=REGION)

body = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 500,
    "messages": [
        {"role": "user", "content": [{"type": "text", "text": prompt}]}
    ]
}

start = time.time()
resp = bedrock_rt.invoke_model(modelId=PROFILE_ARN, body=json.dumps(body))
elapsed = time.time() - start

output = json.loads(resp["body"].read())

reply = output["content"][0]["text"]

# Usage metadata (if present)
usage = output.get("usage", {})
input_tokens = usage.get("input_tokens", "N/A")
output_tokens = usage.get("output_tokens", "N/A")

print("✅ Generated Output:\n")
print(reply)

print("\n📊 Stats:")
print("  Input tokens :", input_tokens)
print("  Output tokens:", output_tokens)
print(f"  Time taken   : {elapsed:.2f} seconds")


✅ Generated Output:

{
  "questions": [
    {
      "question": "What are the key differences between net profit and net present value as project evaluation methods?",
      "difficulty": "Medium",
      "blooms_level": "Analyze" 
    },
    {
      "question": "How does the timing of cash flows impact the financial evaluation of a project?",
      "difficulty": "Medium", 
      "blooms_level": "Understand"
    },
    {
      "question": "What are the advantages and disadvantages of using return on investment (ROI) as a project evaluation technique?",
      "difficulty": "Medium",
      "blooms_level": "Evaluate"
    },
    {
      "question": "Why is it important to consider both the size of investment and timing of returns when evaluating projects?",
      "difficulty": "Easy",
      "blooms_level": "Understand"
    },
    {
      "question": "How can risk factors be incorporated into the financial evaluation of a project?",
      "difficulty": "Hard",
      "blooms_level": "Apply"
 